In [63]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython==8.21.0

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
55 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [98]:
%%manim -ql -v WARNING DijkstraVisualization

from manim import *
import heapq

class DijkstraVisualization(Scene):
    def construct(self):
        # Paleta de Cores
        node_color = BLUE_B
        node_text_color = WHITE
        edge_color = GREY
        path_color = GOLD
        highlight_color = YELLOW

        nodes = {
            "A": LEFT * 3 + UP * 2,
            "B": LEFT * 2 + DOWN * 1,
            "C": LEFT * 1 + UP * 1,
            "D": RIGHT * 1 + UP * 1,
            "E": RIGHT * 2 + DOWN * 1,
            "F": RIGHT * 3 + UP * 2
        }

        node_circles = {
            node: Circle(radius=0.6, color=WHITE, fill_opacity=0.7).move_to(position)
            for node, position in nodes.items()
        }

        node_labels = {
            node: Text(node, color=node_text_color, font_size=30, weight=BOLD).move_to(position)
            for node, position in nodes.items()
        }

        # Centraliza o texto no centro do círculo
        for node, label in node_labels.items():
            label.move_to(node_circles[node].get_center())

        edges = [
            ("A", "B", 4),
            ("A", "C", 2),
            ("B", "C", 5),
            ("B", "D", 10),
            ("C", "D", 3),
            ("C", "E", 8),
            ("D", "F", 7),
            ("E", "F", 4)
        ]

        # Função auxiliar para criar as arestas
        def get_edge(start, end):
            start_pos = nodes[start]
            end_pos = nodes[end]
            return Line(start_pos, end_pos).set_color(edge_color)

        edge_lines = [get_edge(start, end) for start, end, _ in edges]

        edge_labels = {}
        for start, end, weight in edges:
            mid_point = (nodes[start] + nodes[end]) / 2
            label = Text(str(weight), color=WHITE, font_size=24).move_to(mid_point)
            edge_labels[(start, end)] = label

        self.play(
            *[FadeIn(circle) for circle in node_circles.values()],
            *[Write(label) for label in node_labels.values()],
            *[Create(edge) for edge in edge_lines],
            *[Write(label) for label in edge_labels.values()]
        )

        distance_text = Text("Distâncias iniciais:", color=WHITE, font_size=24, weight=BOLD).to_edge(UP)
        self.play(Write(distance_text))

        # Representação do grafo com os pesos das arestas
        graph = {
            "A": [("B", 4), ("C", 2)],
            "B": [("A", 4), ("C", 5), ("D", 10)],
            "C": [("A", 2), ("B", 5), ("D", 3), ("E", 8)],
            "D": [("B", 10), ("C", 3), ("F", 7)],
            "E": [("C", 8), ("F", 4)],
            "F": [("D", 7), ("E", 4)]
        }

        # Função do algoritmo de Dijkstra
        def dijkstra(graph, start):
            distances = {node: float('inf') for node in graph}
            previous_nodes = {node: None for node in graph}
            distances[start] = 0
            visited = set()
            queue = [(0, start)]

            while queue:
                (dist, current_node) = heapq.heappop(queue)
                if current_node in visited:
                    continue
                visited.add(current_node)

                # Atualiza distâncias
                for neighbor, weight in graph[current_node]:
                    alt = dist + weight
                    if alt < distances[neighbor]:
                        distances[neighbor] = alt
                        previous_nodes[neighbor] = current_node
                        heapq.heappush(queue, (alt, neighbor))

            return previous_nodes, distances

        # Executar Dijkstra e obter resultados
        previous_nodes, distances = dijkstra(graph, "A")

        path = []
        node = "F"
        while node != "A":
            path.append(node)
            node = previous_nodes[node]
        path.append("A")
        path = path[::-1]

        # Animar o caminho mais curto com um efeito de destaque suave
        path_lines = [get_edge(path[i], path[i + 1]) for i in range(len(path) - 1)]
        self.play(*[Create(line.set_color(path_color)) for line in path_lines])

        final_node_circle = node_circles["F"]
        self.play(final_node_circle.animate.set_fill(highlight_color, opacity=1).scale(1.2))

        final_text = Text("Caminho mais curto encontrado!", color=WHITE, font_size=30, weight=BOLD).to_edge(DOWN)
        self.play(FadeIn(final_text))  # Aqui, corrigi a animação para FadeIn

        distance_text_2 = Text(f"Distâncias finais: {distances}", color=WHITE, font_size=24).next_to(final_text, UP)
        self.play(Write(distance_text_2))  # Anima a escrita do texto das distâncias finais

        self.wait(3)


Manim Community v0.18.1

In [84]:
!ls media/videos/content/720p30




DijkstraVisualization.mp4  partial_movie_files


In [99]:
from IPython.display import Video

# Caminho para o vídeo gerado
video_path = 'media/videos/content/480p15/DijkstraVisualization.mp4'

# Exibir o vídeo diretamente no Colab com embed
Video(video_path, embed=True)
